In [1]:
import os
import os.path
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold                                                                                                                       
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import sklearn_evaluation.plot as skplot
from sklearn.model_selection import learning_curve

import matplotlib.pyplot as plt

import numpy as np
np.random.seed(1)

from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.vgg19 import VGG19
from keras.applications.imagenet_utils import preprocess_input

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
imagedir = "Datasets/ByFamilyPortexNoLegend"

In [3]:
cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family
for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
    label += 1
num_classes = label

# Compute the features
width, height,channels = (224,224,3)
X = np.zeros((num_samples, width, height, channels))
cnt = 0
list_paths = [] # List of image paths
print("Processing images ...")
for i in range(len(list_fams)):
    for img_file in glob.glob(list_fams[i]+'/*.png'):
        #print("[%d] Processing image: %s" % (cnt, img_file))
        list_paths.append(os.path.join(os.getcwd(),img_file))
        img = image.load_img(img_file, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        X[cnt] = x
        cnt += 1
print("Images processed: %d" %(cnt))

os.chdir(cur_dir)

Label: 0	Family:         allaple	Number of images: 362
Label: 1	Family:           alman	Number of images: 325
Label: 2	Family:          autoit	Number of images: 261
Label: 3	Family:            daws	Number of images: 466
Label: 4	Family:            delf	Number of images: 359
Label: 5	Family:         gamarue	Number of images: 259
Label: 6	Family:          ibryte	Number of images: 347
Label: 7	Family:          loring	Number of images: 285
Label: 8	Family:          mydoom	Number of images: 578
Label: 9	Family:          qukart	Number of images: 253
Label:10	Family:          ramnit	Number of images: 506
Label:11	Family:          sality	Number of images: 1401
Label:12	Family:          simbot	Number of images: 1148
Label:13	Family:       softpulse	Number of images: 912
Label:14	Family:          viking	Number of images: 183
Label:15	Family:         virlock	Number of images: 373
Label:16	Family:          vobfus	Number of images: 405
Label:17	Family:          wapomi	Number of images: 345
Label:18

In [4]:
X.shape

(10136, 224, 224, 3)

In [5]:
# Creating base_model
image_shape = (224, 224, 3)                                                                                                                                                                                                                                                                                            
base_model = VGG19(weights='imagenet', input_shape=image_shape, include_top=False, pooling='avg')

In [6]:
print("Extracting bottleneck features from convolutional layers ...")
bnfeatures = base_model.predict(X)

Extracting bottleneck features from convolutional layers ...


In [7]:
bnfeatures.shape

(10136, 512)

In [8]:
bnfeatures = np.reshape(bnfeatures,(bnfeatures.shape[0],-1))

In [9]:
bnfeatures.shape

(10136, 512)

In [10]:
# Create stratified k-fold subsets                                                                                                                                        
kfold = 10  # no. of folds                                                                 
skf = StratifiedKFold(kfold, shuffle=True,random_state=1)

In [11]:
# Training top_model
C_range = np.logspace(-2, 10, 13)
param_grid = dict(C=C_range)

top_model = GridSearchCV(svm.LinearSVC(), param_grid=param_grid, cv=skf, verbose=10, n_jobs=-1)
top_model.fit(bnfeatures,y)  # Training 

Fitting 10 folds for each of 13 candidates, totalling 130 fits
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   55.7s


[CV] ........................... C=0.01, score=0.937562, total=  55.4s
[CV] C=1.0 ...........................................................
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.937251, total=  55.9s
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.935707, total=  59.7s
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.921260, total= 1.0min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.911417, total=  60.0s
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.939961, total= 1.0min
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.929423, total= 1.0min
[CV] C

[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  1.0min


[CV] ............................ C=1.0, score=0.905273, total= 1.0min
[CV] C=10.0 ..........................................................
[CV] ............................ C=1.0, score=0.915354, total= 1.0min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.911133, total= 1.1min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.919608, total= 1.1min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.924453, total= 1.1min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.926181, total= 1.1min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.924827, total= 1.1min
[CV] C=100.0 .........................................................
[CV] .

...

[CV] ................... C=1000000000.0, score=0.905273, total= 1.2min
[CV] ................... C=1000000000.0, score=0.916339, total= 1.1min
[CV] ................... C=1000000000.0, score=0.928783, total= 1.1min
[CV] ................... C=1000000000.0, score=0.925197, total= 1.1min
[CV] ................... C=1000000000.0, score=0.920588, total= 1.2min
[CV] ................... C=1000000000.0, score=0.913690, total= 1.1min
[CV] ................... C=1000000000.0, score=0.848907, total= 1.1min
[CV] ................... C=1000000000.0, score=0.928642, total= 1.1min
[CV] ................... C=1000000000.0, score=0.935259, total= 1.1min
[CV] .................. C=10000000000.0, score=0.906250, total=  48.5s
[CV] .................. C=10000000000.0, score=0.917808, total=  49.4s
[CV] .................. C=10000000000.0, score=0.914706, total=  49.4s
[CV] .................. C=10000000000.0, score=0.902559, total=  48.5s
[CV] .................. C=10000000000.0, score=0.920870, total=  48.7s
[CV] .

[Parallel(n_jobs=-1)]: Done 125 out of 130 | elapsed:  6.0min remaining:   14.5s


[CV] .................. C=10000000000.0, score=0.896825, total=  47.6s
[CV] .................. C=10000000000.0, score=0.922696, total=  48.8s
[CV] .................. C=10000000000.0, score=0.879722, total=  47.9s
[CV] .................. C=10000000000.0, score=0.928287, total=  48.0s


[Parallel(n_jobs=-1)]: Done 130 out of 130 | elapsed:  6.1min finished


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=1, shuffle=True),
       error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05,
       1.e+06, 1.e+07, 1.e+08, 1.e+09, 1.e+10])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [12]:
top_model.cv_results_

{'mean_fit_time': array([57.41241736, 64.47986686, 64.17425992, 65.55967879, 68.46625228,
        63.37177222, 69.81159379, 65.16005628, 67.4234519 , 67.65743651,
        64.48163531, 67.26666567, 48.57555373]),
 'mean_score_time': array([0.01752439, 0.01621375, 0.0141011 , 0.01121898, 0.01140103,
        0.01013317, 0.0114265 , 0.01111379, 0.01150701, 0.01167645,
        0.01052275, 0.00710459, 0.00537372]),
 'mean_test_score': array([0.92995264, 0.92275059, 0.91554854, 0.91910024, 0.91969219,
        0.91288477, 0.91939621, 0.91979084, 0.92077743, 0.91189818,
        0.9132794 , 0.91574586, 0.91170087]),
 'mean_train_score': array([0.97892024, 0.98650649, 0.97937961, 0.98260433, 0.98454341,
        0.97501869, 0.98358962, 0.98419244, 0.98384113, 0.97160706,
        0.97254028, 0.97388753, 0.9772382 ]),
 'param_C': masked_array(data=[0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0,
                    1000000.0, 10000000.0, 100000000.0, 1000000000.0,
                    1000000

In [13]:
top_model.grid_scores_

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.92995, std: 0.00956, params: {'C': 0.01},
 mean: 0.92275, std: 0.00869, params: {'C': 0.1},
 mean: 0.91555, std: 0.01325, params: {'C': 1.0},
 mean: 0.91910, std: 0.01148, params: {'C': 10.0},
 mean: 0.91969, std: 0.00864, params: {'C': 100.0},
 mean: 0.91288, std: 0.02107, params: {'C': 1000.0},
 mean: 0.91940, std: 0.01182, params: {'C': 10000.0},
 mean: 0.91979, std: 0.00747, params: {'C': 100000.0},
 mean: 0.92078, std: 0.00799, params: {'C': 1000000.0},
 mean: 0.91190, std: 0.02235, params: {'C': 10000000.0},
 mean: 0.91328, std: 0.02301, params: {'C': 100000000.0},
 mean: 0.91575, std: 0.02400, params: {'C': 1000000000.0},
 mean: 0.91170, std: 0.01457, params: {'C': 10000000000.0}]

In [14]:
print("Best parameters: %s - score: %0.4f" % (top_model.best_params_, top_model.best_score_))

Best parameters: {'C': 0.01} - score: 0.9300
